**GIL**: Global Interpreter Lock

GIL prevents python from running threads on different CPU cores.

**Attention**

Even if only one thread runs at a time, a thread's operations on *data structure* can be interrupted between any two butecode instructions in the python interpreter. This is dangerous if we access the same object from different threads in the same time. 

Example:

In [19]:
from threading import Thread, Lock

In [7]:
class Counter(object):
    
    def __init__(self):
        self.count = 0
        
    def increment(self, offset):
        self.count += offset

In [13]:
def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        # do something here
        counter.increment(1)

In [14]:
def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target = func, args=args)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

In [15]:
how_many = 10**5
counter = Counter()
run_threads(worker, how_many, counter)
print("Counter should be %d, found %d" % (5*how_many, counter.count))

Counter should be 500000, found 444858


We are way off here !!!!

The problem comes from the fact that python interpreter enforces fairness between the threads to make sure they all have **the same processing time**. In order to do this, python will suspend a thread that is running and will resume another one. But we never know when python will suspend our threads. A thread can even be paused in the middle of an operation...and that's what happened here !

The *Counter* object's *increment* method is:
```
counter.count += offset
```

But the *+=* operator used on an object attribute acutally instructs phtyon to do 3 separate operations behind the scenes. It is equivalent to:
```
value = getattr(counter, 'count')
result = value + offset
setattr(counter, 'count', result)
```

Python threads incrementing the counter can be suspended between any two of these operations. 

example of what can go wrong:
```
# Running in Thread A
value_a = getattr(counter, 'count')
# Context switch to Thread B
value_b = getattr(counter, 'count')
result_b = value_b + 1
setattr_b(counter, 'count', result_b)
# Context switch back to Thread A
result_a = value_a + 1
setattr(counter, 'count', result_a)
```

Thread A stomped on thread B, erasing all of its progress incrementing the counter !!!!!

### Solution

So to prevent data races like this, Python includes a robust set of tools in the *threading* built-in module.

### lock

This is mutual-exclusion lock (mutex). 

By using a lock, I can have the *Counter* class protec it scurrent value against simultaneous access from multiple threads. Only 1 thread will be able to acquire the lock at a time. 

In [20]:
class LockingCounter(object):
    
    def __init__(self):
        self.lock = Lock()
        self.count = 0
        
    def increment(self, offset):
        with self.lock:
            self.count += offset

In [21]:
counter = LockingCounter()
run_threads(worker, how_many, counter)
print('Counter should be %d, found %d' %(5*how_many, counter.count))

Counter should be 500000, found 500000


###  Things to remember

 * our programs will corrupt data structures if we allow multiple threads to modify the same objects without locks